In [9]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image
import scipy

In [10]:
# Define paths
project_root = os.path.abspath('..')  # Adjust based on your project structure
train_dir = os.path.join(project_root, 'data', 'train')
val_dir = os.path.join(project_root, 'data', 'validation')

In [11]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1280 images belonging to 8 classes.
Found 320 images belonging to 8 classes.


In [13]:
# Build the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
40/40 [==============================] - 29s 644ms/step - loss: 1.6835 - accuracy: 0.3977 - val_loss: 1.3774 - val_accuracy: 0.4938
Epoch 2/20
40/40 [==============================] - 25s 630ms/step - loss: 0.9955 - accuracy: 0.6633 - val_loss: 1.0104 - val_accuracy: 0.6375
Epoch 3/20
40/40 [==============================] - 26s 639ms/step - loss: 0.6827 - accuracy: 0.7953 - val_loss: 0.9567 - val_accuracy: 0.6625
Epoch 4/20
40/40 [==============================] - 26s 636ms/step - loss: 0.5293 - accuracy: 0.8438 - val_loss: 0.8590 - val_accuracy: 0.7156
Epoch 5/20
40/40 [==============================] - 26s 645ms/step - loss: 0.3868 - accuracy: 0.9062 - val_loss: 0.8605 - val_accuracy: 0.7031
Epoch 6/20
40/40 [==============================] - 25s 618ms/step - loss: 0.2936 - accuracy: 0.9406 - val_loss: 0.8466 - val_accuracy: 0.6969
Epoch 7/20
40/40 [==============================] - 25s 623ms/step - loss: 0.2445 - accuracy: 0.9500 - val_loss: 0.8563 - val_accuracy: 0.7000

In [16]:
# Save the model
models_dir = os.path.join(project_root, 'models')
os.makedirs(models_dir, exist_ok=True)
model.save(os.path.join(models_dir, 'gesture_recognition_model.h5'))